In [20]:
import os
import random
import traceback
from pdb import set_trace
import sys
import numpy as np
from abc import ABC, abstractmethod
import traceback

In [21]:
from util.timer import Timer
from util.data import split_data, feature_label_split, Standardization
from util.metrics import mse
from datasets.HousingDataset import HousingDataset

Your Name: Type your name here

# Understand Housing Dataset

![](https://assets.prevu.com/blogs/images/first-time-buyer-boston-real-estate/03d0c13cdf6721a022afd91e343493b5?ixlib=rb-4.0.3&w=670&lossless=true&auto=format%20compress&fit=fill&fill=solid&s=cb885d7fc811865d8d2219c47c87eb01)

The dataset you'll be using for this project is the Boston Housing dataset which contains various different features about houses in Boston. This is a classic machine learning dataset from 1978 and is one of the first datasets most people use when first learning machine learning. **There are 506 samples and 13 feature variables in this dataset. The objective is to predict the value of the house, given by the 'MEDV' column, using the provided features.**

The dataset consists of 3 splits:

1. **Train**: Throughout this assignment you will be training your model using this data.
2. **Validation**: You will then use this set to tune your model and evaluate its performance.
3. **Test**: This split simulates real life data which we often don't have access to until the model is deployed. We have kept this split hidden from you and we will use it to judge the performance of your model.

You DO NOT have access to the Test set as it gonna be used for scoring. This will not prevent you to complete this assignment at all.

The meanings of features and target are listed below. However, the meanings of these attributes will not affect your coding. So read them only if you are interested. 

    1. CRIM      per capita crime rate by town
    2. ZN        proportion of residential land zoned for lots over 
                 25,000 sq.ft.
    3. INDUS     proportion of non-retail business acres per town
    4. CHAS      Charles River dummy variable (= 1 if tract bounds 
                 river; 0 otherwise)
    5. NOX       nitric oxides concentration (parts per 10 million)
    6. RM        average number of rooms per dwelling
    7. AGE       proportion of owner-occupied units built prior to 1940
    8. DIS       weighted distances to five Boston employment centres
    9. RAD       index of accessibility to radial highways
    10. TAX      full-value property-tax rate per 10,000 USD
    11. PTRATIO  pupil-teacher ratio by town
    12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks 
                 by town
    13. LSTAT    % lower status of the population
    14. MEDV     Median value of owner-occupied homes (in 1000 USD)

# Design Machine Learning Models (TODO)

## Base Model
Basic model structure, **don't change** this component.

In [22]:
class BaseModel(ABC):
    """ Super class for ITCS Machine Learning Class"""

    @abstractmethod
    def fit(self, X, y):
        pass

    @abstractmethod
    def predict(self, X):
        pass

## Linear Regression
Basic model structure, **don't change** this component.

In [23]:
class LinearModel(BaseModel):
    """
        Abstract class for a linear model

        Attributes
        ==========
        w       ndarray
                weight vector/matrix
    """

    def __init__(self):
        """
            weight vector w is initialized as None
        """
        self.w = None

    # check if the matrix is 2-dimensional. if not, raise an exception
    def _check_matrix(self, mat, name):
        if len(mat.shape) != 2:
            raise ValueError(f"Your matrix {name} shape is not 2D! Matrix {name} has the shape {mat.shape}")

    # add a biases
    def add_ones(self, X):
        """
            add a column basis to X input matrix
        """
        self._check_matrix(X, 'X')
        return np.hstack((np.ones((X.shape[0], 1)), X))

    ####################################################
    #### abstract funcitons ############################
    @abstractmethod
    def fit(self, X: np.ndarray, y: np.ndarray):
        """
            train linear model

            Args:
                X:  Input data

                y:  targets/labels
        """
        pass

    @abstractmethod
    def predict(self, X: np.ndarray):
        """
            apply the learned model to input X

            parameters
            ----------
            X     2d array
                  input data

        """
        pass

## TODO: Linear Regression with Ordinary Least Square (OLS) 
**Please complete the TODOs. **

In [24]:
class OrdinaryLeastSquares(LinearModel):
    """
        Performs regression using ordinary least squares
        attributes:
            w (np.ndarray): weight matrix

    """

    def __init__(self):
        super().__init__()

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        
        """ Used to train our model to learn optimal weights.

            TODO:
                Finish this method by adding code to perform OLS in order to learn the
                weights `self.w`.
        """
        X_with_bias = self.add_ones(X)
        X_transpose = X_with_bias.T
        self.w = np.linalg.inv(X_transpose @ X_with_bias) @ X_transpose @ y

    def predict(self, X: np.ndarray) -> np.ndarray:
        """ Used to make a prediction using the learned weights.

            TODO:
                Finish this method by adding code to make a prediction given the learned
                weights `self.w`.
        """
        # TODO (REQUIRED) Add code below
        X_with_bias = self.add_ones(X)
        y_hat = X_with_bias @ self.w

        # TODO (REQUIRED) Store predictions below by replacing np.ones()
        #y_hat = np.ones([len(X), 1])

        return y_hat

## TODO: Lienar Regression with least Mean Squares (LMS)
Optimize the model through gradient descent. **Please complete the TODOs. **

In [25]:
class LeastMeanSquares(LinearModel):
    """
        Performs regression using least mean squares (gradient descent)

        attributes:
            w (np.ndarray): weight matrix

            alpha (float): learning rate or step size

            epochs (int): Number of epochs to run for mini-batch
                gradient descent

            seed (int): Seed to be used for NumPy's RandomState class
                or universal seed np.random.seed() function.
    """

    def __init__(self, alpha: float, epochs: int, seed: int = None):
        super().__init__()
        self.w = None
        self.alpha = alpha
        self.epochs = epochs
        self.seed = seed

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """ Used to train our model to learn optimal weights.

            TODO:
                Finish this method by adding code to perform LMS in order to learn the
                weights `self.w`.
        """
        X_with_bias = self.add_ones(X)
        if self.seed is not None:
            np.random.seed(self.seed)
        self.w = np.random.randn(X_with_bias.shape[1], 1)
        for _ in range(self.epochs):
            y_pred = X_with_bias @ self.w
            error = y_pred - y
            gradient = (1/X_with_bias.shape[0]) * (X_with_bias.T @ error)
            self.w = self.w - self.alpha * gradient
    def predict(self, X: np.ndarray) -> np.ndarray:
        """ Used to make a prediction using the learned weights.

            TODO:
                Finish this method by adding code to make a prediction given the learned
                weights `self.w`.
        """
        # TODO (REQUIRED) Add code below
        X_with_bias = self.add_ones(X)
        y_hat = X_with_bias @ self.w

        # TODO (REQUIRED) Store predictions below by replacing np.ones()
        # y_hat = np.ones([len(X), 1])

        return y_hat

## TODO: Polynomial Regression with Ordinary Least Square (OLS) 
**Please complete the TODOs. **

In [26]:
class PolynomialRegression(OrdinaryLeastSquares):
    """
        Performs polynomial regression using ordinary least squares algorithm

        attributes:
            w (np.ndarray): weight matrix that is inherited from OrdinaryLeastSquares

            degree (int): the number of polynomial degrees to include when adding
                polynomial features.
    """

    def __init__(self, degree: int):
        super().__init__()
        self.degree = degree

    def add_polynomial_features(self, X: np.ndarray) -> np.ndarray:
        """ Computes polynomial features given the pass data.

            TODO:
                Finish this method by adding code to compute the polynomial features
                for X. Be sure to return the new data with the polynomial features!

            Hint:
                Feel free to use sklearn.preprocessing.PolynomialFeatures but remember
                it includes the bias so make sure to disable said feature!
        """
        from sklearn.preprocessing import PolynomialFeatures
        poly = PolynomialFeatures(degree=self.degree, include_bias=False)
        X_poly = poly.fit_transform(X)
        
        return X_poly

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """ Used to train our model to learn optimal weights.

            TODO:
                Finish this method by adding code to perform polynomial regression using
                the closed form solution OLS to learn the weights `self.w`.

            Hint:
                Since we inherit from OrdinaryLeastSquares you can simply just call
                super().train(X, y) instead of copying the code from OrdinaryLeastSquares
                after you run self.add_polynomial_features(X).
        """
        X_poly = self.add_polynomial_features(X)
        super().fit(X_poly, y)

    def predict(self, X: np.ndarray) -> np.ndarray:
        """ Used to make a prediction using the learned weights.

            TODO:
                Finish this method by adding code to make a prediction given the learned
                weights `self.w`.

            Hint:
                Since we inherit from OrdinaryLeastSquares you can simply just call
                super().predict(X) instead of copying the code from OrdinaryLeastSquares
                after you run self.add_polynomial_features(X).
        """
        # TODO (REQUIRED) Add code below
        X_poly = self.add_polynomial_features(X)
        y_hat = super().predict(X_poly)

        return y_hat

## TODO: Polynomial Regression with OLS and Regularization
**Please complete the TODOs. **

In [27]:
class PolynomialRegressionRegularized(PolynomialRegression):
    """
        Performs polynomial regression with l2 regularization using the ordinary least squares algorithm
    
        attributes:
            w (np.ndarray): weight matrix that is inherited from OrdinaryLeastSquares
            
            degree (int): the number of polynomial degrees to include when adding
                polynomial features.
    """

    def __init__(self, degree: int, lamb: float):
        super().__init__(degree)
        self.lamb = lamb
        
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """ Used to train our model to learn optimal weights.
        
            TODO:
                Finish this method by adding code to perform polynomial regression using
                the closed form solution OLS with L2 regularization to learn 
                the weights `self.w`.
                
            Hint:
                Add the bias after computing polynomial features. Typically we don't want
                to include the bias when computing polynomial features.
        """
        X_poly = self.add_polynomial_features(X)
    
        X_poly_with_bias = self.add_ones(X_poly)
        n_features = X_poly_with_bias.shape[1]
        reg_matrix = np.eye(n_features)
        reg_matrix[0, 0] = 0  
        
        X_transpose = X_poly_with_bias.T
        self.w = np.linalg.inv(X_transpose @ X_poly_with_bias + self.lamb * reg_matrix) @ X_transpose @ y

    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """ Used to make a prediction using the learned weights.

            TODO:
                This predict() method is exactly the same as the predict() method in the above class `PolynomialRegression`, 
                so you can just simply copy them here. 
        """
        # TODO (REQUIRED) Add code below
        X_poly = self.add_polynomial_features(X)
        y_hat = super().predict(X_poly)
        # TODO (REQUIRED) Store predictions below by replacing np.ones()
        #y_hat = np.ones([len(X), 1])

        return y_hat

# TODO: Define Hyperparameters 

In [28]:
class HyperParameters():
    
    @staticmethod
    def get_params(name):
        model = getattr(HyperParameters, name)
        return {key:value for key, value in model.__dict__.items() 
            if not key.startswith('__') and not callable(key)}
    
    class OrdinaryLeastSquares():
        pass # No hyperparamters to set
        
    class LeastMeanSquares():
        model_kwargs = dict(
            alpha = 0.01, # TODO (REQUIRED) Set your learning rate
            epochs = 1000, # TODO (OPTIONAL) Set number of epochs
            seed = 42, # TODO (OPTIONAL) Set seed for randomly generated weights
        )

        data_prep_kwargs = dict(
            # TODO (OPTIONAL) Set the names of the features/columns to use for the Housing dataset
            use_features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
                            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'],
        )

    class PolynomialRegression():
        model_kwargs = dict(
            degree = 3, # TODO (REQUIRED) Set your polynomial degree
        )
        
    class PolynomialRegressionRegularized():
        model_kwargs = dict(
            degree = 3, # TODO (REQUIRED) Set your polynomial degree
            lamb = 0.1, # TODO (REQUIRED) Set your regularization value for lambda
        )

        data_prep_kwargs = dict(
            # TODO (OPTIONAL) Set the names of the features/columns to use for the Housing dataset
            use_features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
                            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'],
        )

# Model Training and Testing

## Define Preprocessing Functions

In [29]:
def standardize_data(X_trn, X_vld):
    standardize = Standardization()
    X_trn_clean = standardize.fit_transform(X_trn)
    X_eval_clean = standardize.transform(X_vld)
    
    return X_trn_clean, X_eval_clean

def get_cleaned_data(df_trn, df_vld, feature_names, label_name, return_df=False):
    X_trn, y_trn, X_vld, y_vld = split_data(df_trn, df_vld, feature_names, label_name)
    X_trn, X_vld = standardize_data(X_trn, X_vld)

    return X_trn, y_trn, X_vld, y_vld

In [30]:
task_info = [
       dict(
            model=OrdinaryLeastSquares,
            name='OrdinaryLeastSquares',
            threshold=80,
            metrics=dict(MSE=mse),
            eval_metric='MSE',
            trn_score=9999,
            eval_score=9999,
            successful=False,
        ),
        dict(
            model=LeastMeanSquares,
            name='LeastMeanSquares',
            threshold=80,
            metrics=dict(MSE=mse),
            eval_metric='MSE',
            trn_score=9999,
            eval_score=9999,
            successful=False,
        ),
        dict(
            model=PolynomialRegression,
            name='PolynomialRegression',
            threshold=50,
            metrics=dict(MSE=mse),
            eval_metric='MSE',
            trn_score=9999,
            eval_score=9999,
            successful=False,
        ),
        dict(
            model=PolynomialRegressionRegularized,
            name='PolynomialRegressionRegularized',
            threshold=40,
            metrics=dict(MSE=mse),
            eval_metric='MSE',
            trn_score=9999,
            eval_score=9999,
            successful=False,
        )
    ]

## Define Model running (training/fit and testing/evaluate)

In [31]:
def get_name(obj):
    try:
        if hasattr(obj, '__name__'):
            return obj.__name__
        else:
            return obj
    except Exception as e:
        return obj
    
def catch_and_throw(e, err):
    trace = traceback.format_exc()
    print(err + f"\n{trace}")
    raise e

In [32]:
class RunModel():
    t1 = '\t'
    t2 = '\t\t'
    t3 = '\t\t\t'
    def __init__(self, model, model_params):
        self.model_name = model.__name__
        self.model_params = model_params
        self.model = self.build_model(model, model_params)

    def build_model(self, model, model_params):
        print("="*50)
        print(f"Building model {self.model_name}")
        
        try:
            model = model(**model_params)
        except Exception as e:
            err = f"Exception caught while building model for {self.model_name}:"
            catch_and_throw(e, err)
        return model
    
    def fit(self, *args, **kwargs):
        print(f"Training {self.model_name}...")
        print(f"{self.t1}Using hyperparameters: ")
        [print(f"{self.t2}{n} = {get_name(v)}")for n, v in self.model_params.items()]
        try: 
            return self._fit(*args, **kwargs)
        except Exception as e:
            err = f"Exception caught while training model for {self.model_name}:"
            catch_and_throw(e, err)
            
    def _fit(self, X, y, metrics=None, pass_y=False):
        if pass_y:
            self.model.fit(X, y)
        else:
             self.model.fit(X)
        preds = self.model.predict(X)
        scores = self.get_metrics(y, preds, metrics, prefix='Train')
        return scores
    
    def evaluate(self, *args, **kwargs):
        print(f"Evaluating {self.model_name}...")
        try:
            return self._evaluate(*args, **kwargs)
        except Exception as e:
            err = f"Exception caught while evaluating model for {self.model_name}:"
            catch_and_throw(e, err)
        

    def _evaluate(self, X, y, metrics, prefix=''):
        preds = self.model.predict(X)
        scores = self.get_metrics(y, preds, metrics, prefix)      
        return scores
    
    def predict(self, X):
        try:
            preds = self.model.predict(X)
        except Exception as e:
            err = f"Exception caught while making predictions for model {self.model_name}:"
            catch_and_throw(e, err)
            
        return preds
    
    def get_metrics(self, y, y_hat, metrics, prefix=''):
        scores = {}
        for name, metric in metrics.items():
            score = metric(y, y_hat)
            display_score = round(score, 3)
            scores[name] = score
            print(f"{self.t2}{prefix} {name}: {display_score}")
        return scores

In [33]:
def run_eval(eval_stage='validation',task_infos=task_info):
    main_timer = Timer()
    main_timer.start()

    dataset = HousingDataset()
    df_trn, df_vld = dataset.load()

    total_points = 0
      
    for info in task_infos:
        task_timer =  Timer()
        task_timer.start()
        try: 
            params = HyperParameters.get_params(info['name'])
            model_kwargs = params.get('model_kwargs', {})
            data_prep_kwargs = params.get('data_prep_kwargs', {})

            if info['name'] == 'OrdinaryLeastSquares':
                feature_names = "RM"
            elif info['name'] == 'PolynomialRegression':
                feature_names = "LSTAT" 
            else:
                use_features = data_prep_kwargs.get('use_features')
                if use_features is None:
                    err = f"use_features argument for {info['name']} can not be none: received {use_features}"
                    raise ValueError(err)
                elif  len(use_features) < 2 :
                    err = f"use_features argument for {info['name']} must have at least 2 features: received {use_features}"
                    raise ValueError(err)
                
                feature_names = data_prep_kwargs['use_features']

            run_model = RunModel(info['model'], model_kwargs)

            
            X_trn, y_trn, X_vld, y_vld = get_cleaned_data(df_trn, df_vld, feature_names, "MEDV")

            trn_scores = run_model.fit(X_trn, y_trn, info['metrics'], pass_y=True)  # Model training
            eval_scores = run_model.evaluate(X_vld, y_vld, info['metrics'], prefix=eval_stage.capitalize()) # Model testing

            info['trn_score'] = trn_scores[info['eval_metric']]
            info['eval_score'] = eval_scores[info['eval_metric']]
            info['successful'] = True
                
        except Exception as e:
            track = traceback.format_exc()
            print("The following exception occurred while executing this test case:\n", track)
        task_timer.stop()
        
        print("")
        points = rubric_regression(info['eval_score'], info['threshold'])
        print(f"Points Earned: {points}")
        total_points += points

    print("="*50)
    print('')
    main_timer.stop()

    avg_trn_mse, avg_eval_mse, successful_tests = summary(task_info)
    task_eval_mse = get_eval_scores(task_info)
    total_points = int(round(total_points))

    print(f"MSE averages for {successful_tests} successful tests")
    print(f"\tAverage Train MSE: {avg_trn_mse}")
    print("=======================================================")
    print(f"\tAverage {eval_stage.capitalize()} MSE: {avg_eval_mse}")
    
    return (total_points, avg_trn_mse, avg_eval_mse, *task_eval_mse)

## Evaluation Related Functions
Don't change this section.

In [34]:
def rubric_regression(mse, thresh, max_score=20):
    if mse <= thresh:
        score_percent = 100
    elif mse is not None:
        score_percent = (thresh / mse) * 100
        if score_percent < 40:
            score_percent = 40
    else:
        score_percent = 20
    score = max_score * score_percent / 100.0

    return score

def get_eval_scores(task_info):
    return [i['eval_score'] for i in task_info]

def summary(task_info):
    sum_trn_mse = 0
    sum_eval_mse = 0
    successful_tests = 0

    for info in task_info:
        if info['successful']:
            successful_tests += 1
            sum_trn_mse += info['trn_score']
            sum_eval_mse += info['eval_score']
    
    if successful_tests == 0:
        return 9999, 9999, successful_tests
    
    avg_trn_mse = sum_trn_mse / successful_tests
    avg_eval_mse = sum_eval_mse / successful_tests
    return avg_trn_mse, avg_eval_mse, successful_tests

# Test your code
Run the following cell to test your code (or for **debugging**). Currently, the last line of output is "Average Validation MSE: 548.01". The number is super high because the TODOs are still empty (no machine learning model is used) right now. **Please fill in the TODOs through this ipython file and try your best to get a low MSE.** The "Average Validation MSE" will decide your score. 

In [35]:
if __name__ == "__main__":
    run_eval()  

Skipping download. File already exists: /Users/hritz/Downloads/Regression-Assignment/datasets/data/housing.train

Skipping download. File already exists: /Users/hritz/Downloads/Regression-Assignment/datasets/data/housing.val

Skipping download. File already exists: /Users/hritz/Downloads/Regression-Assignment/datasets/data/housing.names

Building model OrdinaryLeastSquares
Training OrdinaryLeastSquares...
	Using hyperparameters: 
		Train MSE: 41.941
Evaluating OrdinaryLeastSquares...
		Validation MSE: 47.325
Elapsed time: 0.0165 seconds

Points Earned: 20.0
Building model LeastMeanSquares
Training LeastMeanSquares...
	Using hyperparameters: 
		alpha = 0.01
		epochs = 1000
		seed = 42
		Train MSE: 21.507
Evaluating LeastMeanSquares...
		Validation MSE: 23.324
Elapsed time: 0.0126 seconds

Points Earned: 20.0
Building model PolynomialRegression
Training PolynomialRegression...
	Using hyperparameters: 
		degree = 3
		Train MSE: 31.826
Evaluating PolynomialRegression...
		Validation MSE: 2

/Users/hritz/Downloads/Regression-Assignment/datasets/HousingDataset.py:79: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_train = pd.read_csv(self.data["paths"]["train"], delim_whitespace=True,
/Users/hritz/Downloads/Regression-Assignment/datasets/HousingDataset.py:82: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_val = pd.read_csv(self.data["paths"]["val"], delim_whitespace=True,


KeyboardInterrupt: 

# Easy Points (20 points)
In case the above coding is too difficult to you guys, here are simple questions which will counts for 20 points (the above coding counts for 80 points) of this assignment. Answer the questions based your understanding of the dataset. Each question worths 2 points. 

Q1: In training dataset, there are how many samples?

A1: 323 

code used- [ if __name__ == "__main__":
    dataset = HousingDataset()
    df_train, df_val = dataset.load()
    print(f"Number of samples in training dataset: {len(df_train)}")]

Q2: In validation dataset, there are how many samples?

A2: 81 

code used- [if __name__ == "__main__":
    dataset = HousingDataset()
    df_train, df_val = dataset.load()
    print(f"Number of samples in validation dataset: {len(df_val)}")]

Q3: In each sample, there are how many input features (or variable)?

A3: Number of input features is 13 and 1 target variable in each sample.

code used- [if __name__ == "__main__":
    dataset = HousingDataset()
    df_train, df_val = dataset.load()
    input_features = len(dataset.data["columns"]) - 1
    print(f"Number of input features: {input_features}")

Q5: What are the input variable (or input features)? Please answer the column names of the input features. Don't include quote mark. 

A5: The input features are:
CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, B, LSTAT

These are the 13 columns used as input features, while MEDV (median value of owner-occupied homes) is the target variable that the model would predict.

Q6: In training datset, what's the mean value of `RM` (Keep two decimal places)? 

A6: RM in training dataset is 6.28

code used- [if __name__ == "__main__":
    dataset = HousingDataset()
    df_train, df_val = dataset.load()
    mean_rm = df_train['RM'].mean()
    print(f"Mean value of RM in training dataset: {mean_rm:.2f}")]

Q7: **After standarldization**, what's the value of `AGE` variable of the 1st sample (the 0-th sample) in training set (Keep two decimal places)? 

A7: Standardized AGE value for first sample: -0.76

code used- [if __name__ == "__main__":
    dataset = HousingDataset()
    df_train, df_val = dataset.load()

    age_mean = df_train['AGE'].mean()
    age_std = df_train['AGE'].std()]

Q8: The output/target variable is 'Continuous' or 'Discrete'?

A8: Continuous

Q9: Will the regression with Ordinary Least Square based on gradient descent or not? Please answer YES or NO.

A9: NO

Q10: In Regularization, increase the coefficient lambda, the model will be more 'Simple' or 'Complex'? 

A10: Simple